In [ ]:
import siibra
from nilearn import plotting
import matplotlib.pyplot as plt
%matplotlib inline 

# Setup `siibra`

### Set a EBRAINS Knowledge Graph access token

`siibra` retrieves some data from the EBRAINS Knowledge Graph, which requires
authentication. To do so, please follow these steps:

 1. If you do not yet have an EBRAINS account, register [here](https://ebrains.eu/register). As you are reading this notebook in the EBRAINS collaboratory, this is most probably not necessary at this point.
 2. Your EBRAINS account needs to be enabled for programmatic access to the EBRAINS Knowledge Graph to fetch metadata. This is formal step to acknowledge additional terms of use, and done quickly by emailing to the KG team. A link and template email to do so can be found right on top of the [Knowledge Graph developer page](https://kg.humanbrainproject.eu/develop.html).
 3. Create an authentication token for EBRAINS by visiting
[the EBRAINS authorization endpoint](https://nexus-iam.humanbrainproject.org/v0/oauth2/authorize). 
 4. Copy the token, and store it in the enviroment variable `HBP_AUTH_TOKEN` (just modify and execute the cell below accordingly).

Note that as of now, you have to get a new token (steps 3. and 4.) approximately every day.

In [ ]:
token = input("Enter your token here, then press 'Enter': ")
siibra.set_ebrains_token(token)


### Adjust the local cache directory if needed

`siibra` maintains a local cache of retrieved data. It will automatically choose the system default of your user account on most common operating systems, but you an also choose an explicit folder by setting the environment variable `SIIBRA_CACHEDIR`. We are not using this here.

In [ ]:
#!mkdir -p /tmp/siibracache
#environ['SIIBRA_CACHEDIR'] = "/tmp/siibracache"
#siibra.clear_cache()

# Accessing parcellation maps

We select the human atlas with cytoarchitectonic maps, and load maps in MNI152 and BigBrain space Maps are returned as `ParcellationMap` objects which include a nibabel `SpatialImage` object as their `.image` attribute for easy use with typical neuroimaging tools. `ParcellationMap` objects can have 4D shape, if the parcellation is split into mutliple maps (e.g. by hemisphere).  The map can be directly iterated to get access to the 3D volumes.

In [ ]:
atlas = siibra.atlases["human"]

In [ ]:
# in MNI 152 space
atlas.select(parcellation="julich")
icbm_map = atlas.get_map(space="mni152")
# the julich brain map comes in separate l/r hemispheres, 
# so we iterate over all maps.
for m in icbm_map.fetchall():
    plotting.plot_stat_map(m)

# bigbrain
reso_mm = 0.64
bigbrain_tpl = atlas.get_template("bigbrain")
bigbrain_map = atlas.get_map(space="bigbrain")
plotting.plot_stat_map(bigbrain_map.fetch(reso_mm),
                       bigbrain_tpl.fetch(reso_mm) )
    
# DK atlas
atlas.select(parcellation="desikan")
dk_map = atlas.get_map(space="mni152")
plotting.plot_stat_map(dk_map.fetch(),cmap=plt.cm.tab10)

# Finding and selection brain regions

`siibra` allows to search for regions through an atlas object in a number of ways. You can use text fragments of region names, region identifiers, or label indices of the parcellation maps. The library does its best to resolve them for you. Let's start with a simple text string:

In [ ]:
atlas.select_parcellation(siibra.parcellations.JULICH_BRAIN_CYTOARCHITECTONIC_MAPS_2_5)
atlas.select_region("v1")

As you see, V1 is subdivided into hemispheres. We can be more specific and select the left one. This is a leaf of the region hierarchy tree.

In [ ]:
atlas.select_region("v1 left")

In order to specifiy an element right away, we can use the 'regionnames' glossary stored in the atlas:

In [ ]:
atlas.select_region(atlas.regionnames.AREA_HOC1_V1_17_CALCS_LEFT)

Of course, we can also search for regions, even across all known parcellations:

In [ ]:
matches = atlas.find_regions('ventral',all_parcellations=True)
for region in matches:
    print("{}: {} (index: {})".format(region.parcellation.name,region.name,region.index))

Regions which are labelleled in the parcellation map have a valid `labelindex` attached to them. We can also use this index to search and select regions:

In [ ]:
atlas.select_region(112)

`siibra` supports access to continuous maps for some parcellations. For the Julich-Brain, this translates to the actual probability maps of each area. Let's look at a probability map of the frontal pole in the ICBM space.

In [ ]:
fp2r = atlas.select_region(region='fp2 right')
pmap = fp2r.get_regional_map("mni152", siibra.MapType.CONTINUOUS)
plotting.plot_stat_map(pmap.fetch())

# Extracting multimodal data features

`siibra` provides access to data features of different modalities using the `get_features` method, which accepts a feature modality and is sensitive to the selections configured in the atlas (parcellation, region). If not particular selection is made, `get_features` considers all brain regions of the current parcellation.

### Extract transmitter receptor densities

Transmitter receptor density fingerprints are linked to brain regions by their name in the EBRAINS Knowledge Graph. Like any data feature, they are accessed using the `get_features` method of the atlas, which makes use of the current selection in the atlas. The `get_features` method knows from the specified data modality that the match is determined from the brain region identified. Receptor densities come as a nicely structured datatype. Amongst other things, they can visualize themselves in a plot.

If we don't specify a particular selection, the atlas will return all available receptor density features linked to the parcellation map:

In [ ]:
atlas.clear_selection()
features = atlas.get_features(
    siibra.modalities.ReceptorDistribution)
print("Receptor density features found for the following regions:")
print(", ".join({f.regionspec for f in features}))

If we select a region, the returned list is filtered accordingly. 

In [ ]:
atlas.select_region("V1")
features = atlas.get_features(
    siibra.modalities.ReceptorDistribution)
for r in features:
    fig = r.plot(r.regionspec)

### Extracting Gene Expressions from the Allen Atlas 

The atlas client can make calls to gene expression data from the Allen atlas and evaluate them in the ICBM space to find regional gene expression levels. It also has a list of available gene names for convenient selection. Gene expressions are linked to atlas regions by coordinates of their probes in MNI space. The `get_features` method detects this from the feature modality, and applies the mask of the regions that are currently selected in the atlas to filter the probes. We can visualize these filtered locations.

In [ ]:
atlas.select_region("V1")
features = atlas.get_features(
    siibra.modalities.GeneExpression, 
    gene=siibra.features.gene_names.GABARAPL2)
print(features[0])

# plot
all_coords = [tuple(g.location) for g in features]
mask = atlas.build_mask(siibra.spaces.MNI152_2009C_NONL_ASYM)
display = plotting.plot_roi(mask)
display.add_markers(all_coords,marker_size=5) 

### Retrieving a connectivity matrix
For `siibra`, a connectivity matrix is a data feature like the others below, and can be found using the same `get_features` function just by choosing another modality. `siibra` knows from the modality type that this type of data does not match to the selected brain region in the atlas, but to the selected parcellation. Therefore, `get_features` return connectivity datasets that are defined for the selected parcellation. 

To learn about the nature of the provided connectivity, the `src_info` attribute provides a detailed description of the dataset. 

In [ ]:
# Get the first four connectivity matrices available for the parcellation
features = atlas.get_features(siibra.modalities.ConnectivityMatrix)[:4]

# format dataset names for use as figure titles
from textwrap import wrap
titleformat = lambda text : "\n".join(wrap(text.replace('_',' '),20)) 

# plot the matrices
f,axs = plt.subplots(1,len(features))
for i,feature in enumerate(features):
    axs[i].imshow(feature.array,cmap=plt.cm.viridis)
    axs[i].set_title(titleformat(feature.name),size=8)
f.show()

### Retrieving regional datasets from EBRAINS

In [ ]:
atlas.select_region("V1")
features = atlas.get_features(siibra.modalities.EbrainsRegionalDataset)
for f in features:
    print(f.name)